In [9]:
import pandas as pd
import numpy as np

In [10]:
def convert(series):
    l = []
    for s in series:
        j=0
        while j<5:
            if j==2:
                s = s[:s.index(".")]+" "+s[s.index(".")+1:]
            elif j==3 or j==4:
                s = s[:s.index(".")]+":"+s[s.index(".")+1:]
            else:
                s = s[:s.index(".")]+"/"+s[s.index(".")+1:]
            j+=1
        l.append(s)

    return pd.to_datetime(pd.Series(l), dayfirst=True)

In [11]:
def generate_webpage(country, values):
    f = open("pages/page_INDIA.html")
    lines = f.readlines()
    for i in range(len(lines)):
        if "INDIA" in lines[i]:
            a = lines[i]
            lines[i] = a[:a.find("INDIA")] + "{}".format(country) + a[a.find("INDIA")+len("INDIA"):]
        
        if 'h3 style="color:#ff0007;">' in lines[i]:
            b = lines[i]
            lines[i] = b[:b.find('h3 style="color:#ff0007;">')+26] + "{}".format(values[0]) + b[b.find('</h3>'):]
            
        if 'h3 style="color:#585957;">' in lines[i]:
            b = lines[i]
            lines[i] = b[:b.find('h3 style="color:#585957;">')+26] + "{}".format(values[1]) + b[b.find('</h3>'):]
            
        if 'h3 style="color:#008d1f;">' in lines[i]:
            b = lines[i]
            lines[i] = b[:b.find('h3 style="color:#008d1f;">')+26] + "{}".format(values[2]) + b[b.find('</h3>'):]
        
    f.close()
    f = open("pages/page_{}.html".format(country), "w+")
    s = ""
    for i in lines:
        s+=i
    f.write(s)
    f.close()


In [12]:
def update_homepage():
    df_time = df.copy()
    df_time = df_time[df_time["Time"]==np.max(df_time["Time"])]
    values = (np.sum(df_time["Confirmed Case"]), np.sum(df_time["Death"]), np.sum(df_time["Recover"]))
    f = open("index.html", "r+")
    lines = f.readlines()
    for i in range(len(lines)):
        if 'h3 style="color:#ff0007;">' in lines[i]:
            b = lines[i]
            lines[i] = b[:b.find('h3 style="color:#ff0007;">')+26] + "{}".format(values[0]) + b[b.find('</h3>'):]
            
        if 'h3 style="color:#585957;">' in lines[i]:
            b = lines[i]
            lines[i] = b[:b.find('h3 style="color:#585957;">')+26] + "{}".format(values[1]) + b[b.find('</h3>'):]
            
        if 'h3 style="color:#008d1f;">' in lines[i]:
            b = lines[i]
            lines[i] = b[:b.find('h3 style="color:#008d1f;">')+26] + "{}".format(values[2]) + b[b.find('</h3>'):]
            
    s = ""
    for i in lines:
        s+=i
    f.seek(0)
    f.write(s)
    f.close()

In [13]:
df = pd.read_csv("data_csv/master1.csv")
df["Time"] = convert(df["Time"])
df = df.sort_values(["Nation", "timestamp"]).set_index("Nation")
df.head()

,Time,Confirmed Case,Death,Recover,timestamp
Nation,,,,,
AFGHANISTAN,2020-04-03 14:30:10,273.0,7.0,10.0,8260210
AFGHANISTAN,2020-04-03 14:30:11,273.0,7.0,10.0,8260211
AFGHANISTAN,2020-04-03 17:30:12,273.0,7.0,10.0,8271012
AFGHANISTAN,2020-04-03 20:30:12,273.0,7.0,10.0,8281812
AFGHANISTAN,2020-04-03 23:30:19,281.0,7.0,10.0,8292619


In [14]:
df.to_csv("data_csv/by_country/total.csv", index=True)

In [15]:
for country in df.index.unique():
    df_x = df.loc[country]
    df_x.to_csv("data_csv/by_country/{}.csv".format(country), index=True)
    
    df_x = df_x[df_x["Time"]==np.max(df_x["Time"])]
    values = (df_x["Confirmed Case"][0], df_x["Death"][0], df_x["Recover"][0])
    
    generate_webpage(country, values)

In [8]:
update_homepage()